In [1]:
import os
import pandas as pd
import numpy as np
import gc

In [2]:
os.chdir("C:/Users/shaur/Downloads/commonlit")
os.getcwd()

'C:\\Users\\shaur\\Downloads\\commonlit'

In [3]:
prompts_train = pd.read_csv("prompts_train.csv")
prompts_test = pd.read_csv("prompts_test.csv")

summaries_train = pd.read_csv("summaries_train.csv")
summaries_test = pd.read_csv("summaries_test.csv")

sample_submission = pd.read_csv("sample_submission.csv")

prompts_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [4]:
train = summaries_train.merge(prompts_train, how="left", on="prompt_id")
test = summaries_test.merge(prompts_test, how="left", on="prompt_id")

In [5]:
import warnings
import logging
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset, load_dataset, load_from_disk, load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupShuffleSplit
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


c:\Users\shaur\anaconda3\envs\nlp_start\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
disable_progress_bar()

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

In [8]:
def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)

    return (content_score + wording_score)/2

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(seed=42)

In [9]:
splitter = GroupShuffleSplit(test_size=.2, n_splits=4, random_state=42)
split = splitter.split(train, groups=train['prompt_id'])
train_ind, val_ind = next(split)

train_split = train.iloc[train_ind]
val_split = train.iloc[val_ind]

In [10]:
train_split.prompt_id.value_counts()

prompt_id
39c16e    2057
ebad26    1996
814d6b    1103
Name: count, dtype: int64

In [11]:
val_split.prompt_id.value_counts()

prompt_id
3b9047    2009
Name: count, dtype: int64

In [12]:
train_content = train_split[["text", "content", "wording"]]
val_content = val_split[["text", "content", "wording"]]
test_ = test[["text"]]

In [13]:
gc.collect()

20

In [14]:
model_name = "deberta_v3_base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
seed_everything(seed=42)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

In [15]:
num_layers_to_freeze=20
layer_count=0

for name, param in model.named_parameters():
    if layer_count<num_layers_to_freeze:
        param.requires_grad=False
        layer_count+=1
    else: break

In [16]:
train_dataset_content = Dataset.from_pandas(train_content, preserve_index=False)
val_dataset_content = Dataset.from_pandas(val_content, preserve_index=False)
test_dataset = Dataset.from_pandas(test_, preserve_index=False)

In [17]:
def tokenize_function(examples):
    labels = [examples["content"], examples["wording"]]
    tokenized = tokenizer(examples["text"],
                         padding=False,
                         truncation=True,
                        )
    return {
        **tokenized,
        "labels": labels,
        }

def tokenize_function_test(examples):
    tokenized = tokenizer(examples["text"],
                         padding=False,
                         truncation=True,
                         )
    return tokenized

train_tokenized_datasets_content = train_dataset_content.map(tokenize_function, batched=False)
val_tokenized_datasets_content = val_dataset_content.map(tokenize_function, batched=False)
test_tokenized_dataset = test_dataset.map(tokenize_function_test, batched=False)


In [18]:
model_dir = f"./Results/{model_name}_results"
os.makedirs(model_dir, exist_ok=True)

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
training_args = TrainingArguments(
    output_dir = model_dir,
    load_best_model_at_end = True,
    learning_rate = 1.5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    greater_is_better=False,
    evaluation_strategy="epoch",
    metric_for_best_model="mcrmse",
    save_total_limit=4
)

trainer_content = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized_datasets_content,
        eval_dataset=val_tokenized_datasets_content,
        tokenizer=tokenizer,
        compute_metrics=compute_mcrmse,
        data_collator=data_collator
    )

In [21]:
trainer_content.train()

  6%|▋         | 502/7734 [00:45<10:11, 11.83it/s]

{'loss': -4.5993, 'learning_rate': 1.4030256012412724e-05, 'epoch': 0.19}


 13%|█▎        | 1002/7734 [01:31<14:04,  7.97it/s]

{'loss': -17.9247, 'learning_rate': 1.3060512024825446e-05, 'epoch': 0.39}


 19%|█▉        | 1502/7734 [02:22<09:05, 11.43it/s]

...

                                                   
100%|██████████| 7734/7734 [14:11<00:00,  6.83it/s]

{'eval_loss': -312.5356750488281, 'eval_content_rmse': 284.9277648925781, 'eval_wording_rmse': 290.3125305175781, 'eval_mcrmse': 287.6201477050781, 'eval_runtime': 36.3088, 'eval_samples_per_second': 55.331, 'eval_steps_per_second': 27.679, 'epoch': 3.0}


100%|██████████| 7734/7734 [14:13<00:00,  9.06it/s]

{'train_runtime': 853.6212, 'train_samples_per_second': 18.12, 'train_steps_per_second': 9.06, 'train_loss': -179.15861548716504, 'epoch': 3.0}


TrainOutput(global_step=7734, training_loss=-179.15861548716504, metrics={'train_runtime': 853.6212, 'train_samples_per_second': 18.12, 'train_steps_per_second': 9.06, 'train_loss': -179.15861548716504, 'epoch': 3.0})

In [22]:
best_check = os.listdir(model_dir)[0]

model_best = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{best_check}")
model_best.eval()

test_args = TrainingArguments(
    output_dir=model_dir,
    do_train=False,
    do_predict=True,
    per_device_eval_batch_size=4,
    dataloader_drop_last=False
)

infer_content = Trainer(
    model=model_best,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=test_args
)

val_results_content = infer_content.predict(val_tokenized_datasets_content)[0]
test_results_content = infer_content.predict(test_tokenized_dataset)[0]

model_best.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

100%|██████████| 1/1 [00:00<?, ?it/s]


('./Results/deberta_v3_base_results\\tokenizer_config.json',
 './Results/deberta_v3_base_results\\special_tokens_map.json',
 './Results/deberta_v3_base_results\\spm.model',
 './Results/deberta_v3_base_results\\added_tokens.json',
 './Results/deberta_v3_base_results\\tokenizer.json')

In [33]:
cv_metric = compute_mcrmse((val_results_content, val_content[["content", "wording"]]))
print(f"cv mcrmse: {cv_metric}")

cv mcrmse: {'content_rmse': 99.86622475154807, 'wording_rmse': 104.12056535539071, 'mcrmse': 101.99339505346938}


In [35]:
test_results_content

array([[-147.70258,  152.60568],
       [-147.68437,  152.58734],
       [-147.7017 ,  152.605  ],
       [-147.69392,  152.59676]], dtype=float32)